In [1]:
import sys
import numpy as np
from PyQt5.QtWidgets import (
    QApplication,
    QMainWindow,
    QLabel,
    QAction,
    QFileDialog,
    QWidget,
    QHBoxLayout,
)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap, QImage
import image_tools

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        
        self.setWindowTitle("영상처리")
        self.setGeometry(100, 300, 800, 600)
        
        # 중앙 위젯 설정: QHBoxLayout으로 두 이미지 나란히 표시
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout = QHBoxLayout(self.central_widget)
        self.layout.setContentsMargins(10, 10, 10, 10)
        
        # 원본 이미지 라벨
        self.image_label = QLabel("이미지", self)
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setMinimumSize(200, 200)
        self.layout.addWidget(self.image_label)
        self.layout.setStretch(0, 1)
        
        # 변환된 이미지 라벨
        self.transformed_label = QLabel("변환된 이미지", self)
        self.transformed_label.setAlignment(Qt.AlignCenter)
        self.transformed_label.setMinimumSize(200, 200)
        self.layout.addWidget(self.transformed_label)
        self.layout.setStretch(1, 1)
        
        # 초기 이미지 로드
        self.img = image_tools.load_image('forest.jpg')
        if self.img is not None:
            self.img = np.ascontiguousarray(self.img[:, :, ::-1], dtype=np.uint8)  # BGR to RGB
        else:
            self.image_label.setText("이미지 로드 실패")
            self.transformed_label.setText("이미지 없음")
       
        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()
        file_menu = menubar.addMenu("파일")
        open_action = QAction("열기", self)
        open_action.triggered.connect(self.open_image_dialog)
        file_menu.addAction(open_action)
        exit_action = QAction("종료", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)
        
        transform_menu = menubar.addMenu("변환")
        log_action = QAction("로그 변환", self)
        log_action.triggered.connect(self.apply_log_transform)
        transform_menu.addAction(log_action)
        gamma_action = QAction("감마 변환", self)  # 감마 변환 추가
        gamma_action.triggered.connect(self.apply_gamma_transform)
        transform_menu.addAction(gamma_action)

    def update_image(self):
        if self.img is not None:
            h, w, ch = self.img.shape
            if ch != 3:
                self.image_label.setText("오류: RGB 이미지만 지원됩니다")
                return
            bytes_per_line = ch * w
            self.img = np.ascontiguousarray(self.img, dtype=np.uint8)
            qimg = QImage(self.img.data, w, h, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qimg)
            label_size = self.image_label.size()
            self.image_label.setPixmap(pixmap.scaled(
                label_size, Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.image_label.setText("이미지 로드 실패")

    def update_transformed_image(self):
        if hasattr(self, 'transformed_img') and self.transformed_img is not None:
            h, w, ch = self.transformed_img.shape
            if ch != 3:
                self.transformed_label.setText("오류: RGB 이미지만 지원됩니다")
                return
            bytes_per_line = ch * w
            self.transformed_img = np.ascontiguousarray(self.transformed_img, dtype=np.uint8)
            qimg = QImage(self.transformed_img.data, w, h, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qimg)
            label_size = self.transformed_label.size()
            self.transformed_label.setPixmap(pixmap.scaled(
                label_size, Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
        else:
            self.transformed_label.setText("변환된 이미지 없음")

    def showEvent(self, event):
        super().showEvent(event)
        self.update_image()
        self.update_transformed_image()

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.update_image()
        self.update_transformed_image()

    def open_image_dialog(self):
        filename, _ = QFileDialog.getOpenFileName(
            self,
            "이미지 열기",
            "",
            "Image Files (*.png *.jpg *.jpeg *.bmp *.tif *.tiff)"
        )
        if filename:
            image = image_tools.load_image(filename)
            if image is not None:
                self.img = np.ascontiguousarray(image[:, :, ::-1], dtype=np.uint8)
                self.update_image()
                self.transformed_label.clear()
            else:
                self.image_label.setText("이미지 로드 실패")
                self.transformed_label.setText("이미지 없음")

    def apply_log_transform(self):
        def log_transformation(image, f_max=255):
            f_max = float(f_max)
            if f_max <= 0 or np.isnan(f_max) or np.isinf(f_max):
                f_max = 255
            image = image.astype(np.float32)
            image = np.clip(image, 0, 255)
            C = 255 / np.log(1 + f_max + 1e-10)
            return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

        if self.img is None:
            self.transformed_label.setText("이미지 없음")
            return

        transformed = log_transformation(self.img, f_max=self.img.max())
        self.transformed_img = transformed
        self.update_transformed_image()

    def apply_gamma_transform(self):
        def gamma_transformation(image, gamma=0.5):
            image = image.astype(np.float32)
            image = np.clip(image, 0, 255)  # 값 범위 제한
            normalized = image / 255.0  # 0~1로 정규화
            transformed = np.power(normalized, gamma) * 255.0  # 감마 변환
            return transformed.round().clip(0, 255).astype(np.uint8)

        if self.img is None:
            self.transformed_label.setText("이미지 없음")
            return

        transformed = gamma_transformation(self.img)
        self.transformed_img = transformed
        self.update_transformed_image()

app = QApplication.instance()
if app is None:
    app = QApplication([])
viewer = BasicViewer()
viewer.show()
try:
    sys.exit(app.exec_())
except SystemExit:
    pass